# This notebook parses the Annual Wellness Visits
#  Summary by Provider
##### Added Excel Automation including Filtering, Styling, Security, Pre-filtering 
#### Joe Eberle - 7/25/2022 
#### Anna Mucha - 8/16/2022


In [1]:
# Import Libraries for Spreadsheet Automation 
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date
import random
import math
import os
import glob
from openpyxl import load_workbook
from openpyxl.styles import colors
from openpyxl.utils import get_column_letter
from openpyxl.styles import NamedStyle, Font, Border, Side, PatternFill, GradientFill, Alignment, Color, Fill
from openpyxl import Workbook
import openpyxl
import time

In [2]:
# Establish constants and variables for visual consistency
LIGHT_BLUE = "C5D9F1"
YELLOW = "FFFF00"
GRAY = "E4DFEC"
DARK_BLUE = "1072BA"
DEEP_BLUE = "1072BA"
WHITE = "FFFFFF"
GREEN = "42FF33"
PURPLE = 'FC33FF'
PINK = 'F3C4E6'
DARK_GREEN = '23C632'
BABY_BLUE = 'A7EBE8'
RED = 'F52C26'

CONTENT_TAB_COLOR = DEEP_BLUE
USER_TAB_COLOR = YELLOW
BLANK_WHITE = WHITE 
HEADER_COLOR = LIGHT_BLUE  # Light Blue - Light Blue indicates relevant content for validation 
HEADER_COLOR_USER_ENTRY = YELLOW # Yellow - Yellow indicates a field for User Entry 
HEADER_COLOR_INFORMATIONAL_ONLY = GRAY # Gray - Indicates a field that maybe helpful but for information purposes only 

In [3]:
# Configure the code and visual settings to have consistency - Kaleida PC Share files 
path = 'J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Annual_Wellness_Visits/'
Path_to_Excel = 'J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Annual_Wellness_Visits/'
Path_to_Save_Excel = 'J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Annual_Wellness_Visits_ft/'
Path_to_Publish_Excel = 'J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Annual_Wellness_Visits_pb/'
Path_to_Raw_Data = 'J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/'

Raw_Data_File_Name_1 = 'Quality_ScoreCards_AWV_Summary_By_Provider.xlsx'
Raw_Data_File_Name_2 = 'Quality_ScoreCards_AWV_Summary_By_Provider_Treated.xlsx'
Raw_Data_File_Name_3 = 'Quality_Scorecards_AWV_by_Organization_Class.xlsx'
Raw_Data_File_Name_4 = 'Quality_Scorecards_AWV_Summary_by_Organization.xlsx'
 
logo_file = 'IT-Data-Services-Logo-Color.png'
print(Path_to_Raw_Data + Raw_Data_File_Name_1)
print(Path_to_Raw_Data + Raw_Data_File_Name_2)
print(Path_to_Raw_Data + Raw_Data_File_Name_3)
print(Path_to_Raw_Data + Raw_Data_File_Name_4)

J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Quality_ScoreCards_AWV_Summary_By_Provider.xlsx
J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Quality_ScoreCards_AWV_Summary_By_Provider_Treated.xlsx
J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Quality_Scorecards_AWV_by_Organization_Class.xlsx
J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Quality_Scorecards_AWV_Summary_by_Organization.xlsx


In [4]:
# Read in the excel for provider  
Provider_panel_treated = pd.read_excel(Path_to_Raw_Data + Raw_Data_File_Name_2)
Provider_panel_treated.rename(str.title, axis='columns', inplace = True )
Provider_panel_treated.rename(columns={'Baseline_Percentage' : 'Goal'}, inplace = True)

Provider_panel_treated.head(2)

,Organization_Class,Organization_Name,Provider_Name,Number_Of_Patients,Awv_Met_Count_Numerator,Awv_Not_Met_Count_Denominator,Goal,Percent_Completed,Differential
0,General Physician PC,GPPC PCP Amherst Sheridan,"Cange, Jean",2713,941,1892,85,35,50
1,General Physician PC,GPPC PCP Amherst Sheridan,"Mafi, Ahmad",1287,425,865,85,33,52


In [5]:
# Read in the CSV SQL output for Patient Attribution 
patient_panel = pd.read_excel(Path_to_Raw_Data + Raw_Data_File_Name_1)
patient_panel.rename(str.title, axis='columns', inplace = True )
patient_panel.rename(columns={'Baseline_Percentage_%' : 'Goal'}, inplace = True)

patient_panel.head(2)

,Organization_Class,Organization_Name,Provider_Name,Age_Categories,Number_Of_Patients,Awv_Met_Count_Numerator,Awv_Not_Met_Count_Denominator,Goal,Percent_Completed_%,Differential
0,General Physician PC,GPPC PCP Amherst Sheridan,"Cange, Jean",(0-17 yrs),16,6,10,85,38,48
1,General Physician PC,GPPC PCP Amherst Sheridan,"Cange, Jean",(18-64 yrs),1647,654,1039,85,40,45


In [6]:
Organizational_panel = pd.read_excel(Path_to_Raw_Data + Raw_Data_File_Name_4)
Organizational_panel.rename(str.title, axis='columns', inplace = True )
Organizational_panel.rename(columns={'Baseline_Percentage' : 'Goal'}, inplace = True)

Organizational_panel.head()

,Organization_Class,Organization_Name,Number_Of_Patients,Awv_Met_Count_Numerator,Awv_Not_Met_Count_Denominator,Goal,Percent_Completed,Differential
0,General Physician PC,GPPC PCP Amherst Sheridan,6911,2451,4757,85,35,50
1,General Physician PC,GPPC PCP Buffalo Main,13604,5124,9701,85,38,47
2,General Physician PC,GPPC PCP Depew Transit,8407,3017,6410,85,36,49
3,General Physician PC,GPPC PCP Dunkirk Vineyard Ste 1,6799,2135,4988,85,31,54
4,General Physician PC,GPPC PCP East Aurora Main,1585,633,1104,85,40,45


In [7]:
Organizational_class_panel = pd.read_excel(Path_to_Raw_Data + Raw_Data_File_Name_3)
Organizational_class_panel.rename(str.title, axis='columns', inplace = True )
Organizational_class_panel.rename(columns={'Baseline_Percentage' : 'Goal'}, inplace = True)

Organizational_class_panel.head()

,Organization_Class,Number_Of_Patients,Awv_Met_Count_Numerator,Awv_Not_Met_Count_Denominator,Goal,Differential,Percent_Completed
0,General Physician PC,75456,32922,64973,85,41,44%


In [8]:
# Trim Time from Birth Date and Add Additional fields to support User Validation 
# You cannot run this twice 
#patient_panel['Birth_Date'] = patient_panel['Birth_Date'].dt.date 
#patient_panel['Last_AWV_Date'] = patient_panel['Last_AWV_Date'].dt.date 
#patient_panel['Most_Recent_Date_of_Service'] = patient_panel['Most_Recent_Date_of_Service'].dt.date 
#patient_panel['Deceased_Date'] = patient_panel['Deceased_Date'].dt.date 
#patient_panel['Birth_Date'] = patient_panel['Birth_Date'].dt.date 
#patient_panel['Is Patient cared for by Organization (Y/N) ?'] = ' '
#patient_panel['Is Patient attributed to correct PCP (Y/N) ?'] = ' '
#patient_panel['Optional Comments'] = ' '
#patient_panel['Flagged for Test (Y/N)'] = 'N'
#patient_panel['Is Patient info Correct? (Y/N)'] = ' '
#patient_panel['Is Patient Wellness Visit Correctly Documented (Y/N) ?'] = ' '

In [9]:
# The following function creates patient panels based upon provider name 
def create_provider_patient_panel(Provider_Name):
    Provider_panel = patient_panel[(patient_panel.Provider_Name == Provider_Name)]    
    patient_rows = len(Provider_panel) - 1 
    if patient_rows > 1:
        spread_name = Provider_panel['Provider_Name'].iloc[0]
        csv_name = Provider_panel['Provider_Name'].iloc[0]
        if spread_name == '':
            spread_name = 'No_Provder_NAME_' 
        if csv_name == '':
            csv_name = 'No_Provder_NAME_'              
        spread_name = spread_name.replace('/','')  
        spread_name = spread_name.replace(',','')  # Eliminate commas from provider names
        csv_name = csv_name.replace('/','')     
        csv_name = csv_name.replace(',','')    # Eliminate commas from provider names                
    #    lesser = min(patient_rows,25)
     #   for x in range(lesser):
      #      Provider_panel.iat[random.randint(0,patient_rows),11] = 'Y'
       # spath = 'c:/Users/josep/Documents/Kaleida/Validation/'
        spath = Path_to_Excel
        spread_name = spath + spread_name.replace(' ','_') + '.xlsx'
        csv_name = spath + csv_name.replace(' ','_') + '.csv'
     #   spread_name = spread_name.replace(' ','')
     #   csv_name = csv_name.replace(' ','')        
        Provider_panel.to_excel(spread_name,sheet_name='Patient AWV', index = False)  
        Provider_panel.to_csv(csv_name, index = False)          
        print(spread_name)
        return Provider_Name

In [10]:
# Chunck the data into provider level panels
Provider_Names = patient_panel.Provider_Name.unique()
for Provider in Provider_Names:
    create_provider_patient_panel(Provider)

J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Annual_Wellness_Visits/Cange_Jean.xlsx
J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Annual_Wellness_Visits/Mafi_Paisley.xlsx
J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Annual_Wellness_Visits/Baez_Maritza.xlsx
J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Annual_Wellness_Visits/Charles_Richard.xlsx
J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Annual_Wellness_Visits/Cleary_Kevin.xlsx
J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Annual_Wellness_Visits/Kuehnling_William.xlsx
J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Annual_Wellness_Visits/Lee_Tat-Sum.xlsx
J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Annual_Wellness_Visits/Mitchell_Maxim.xlsx
J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Annual_Wellness_Visits/Lehmann_Leslie.xlsx
J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Annual_Wellness_Visits/Figueroa_

In [11]:
# Add the additional summary tabs for org class, organization, and Provider Treated lives
extension = 'xlsx'
print(path)
os.chdir(path)

files = glob.glob('*.{}'.format(extension))
 
for x in range(len(files)): 
#    print(files[x]) 
    file_path = Path_to_Excel + files[x]
    #org_name =  files[x].replace('.xlsx','')
    #org_name =  org_name.replace('_',' ')    
   #print (org_name)
    #Org_Summary = Organization_Summary[(Organization_Summary.Organization_Name == org_name)]  
    #Org_Summary = Org_Summary.reset_index(drop=True)
    print (file_path)
    writer = pd.ExcelWriter(file_path, engine='openpyxl', mode='a')
    writer.book = load_workbook(file_path)
    Organizational_class_panel.to_excel(writer,sheet_name='Organization Class Summary', index = False) 
    Organizational_panel.to_excel(writer,sheet_name='Organization Summary', index = False)    
    Provider_panel_treated.to_excel(writer,sheet_name='Provider Treated Lives', index = False)
   
    writer.save()
    writer.close()

J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Annual_Wellness_Visits/
J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Annual_Wellness_Visits/Cange_Jean.xlsx
J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Annual_Wellness_Visits/Mafi_Paisley.xlsx
J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Annual_Wellness_Visits/Baez_Maritza.xlsx
J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Annual_Wellness_Visits/Charles_Richard.xlsx
J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Annual_Wellness_Visits/Cleary_Kevin.xlsx
J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Annual_Wellness_Visits/Kuehnling_William.xlsx
J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Annual_Wellness_Visits/Lee_Tat-Sum.xlsx
J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Annual_Wellness_Visits/Mitchell_Maxim.xlsx
J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Annual_Wellness_Visits/Lehmann_Leslie.xlsx
J:/OPA/G

In [26]:
#provider by age group 
def prettify_sheet_one(work_book):
    
# Set the TAB and color it              
    ws1 = work_book.active
    ws1.title = "Annual Wellness by Age"
    ws1.sheet_properties.tabColor = CONTENT_TAB_COLOR 
    
    
# Format the Header as text bold and backgound light blue centered wordwrap     
    max_cols = ws1.max_column + 1 
    for col in range(1,max_cols):
        ws1[get_column_letter(col) + '1'].font = Font(bold=True ) 
        ws1[get_column_letter(col) + '1'].fill = PatternFill("solid", start_color=HEADER_COLOR) 
        ws1[get_column_letter(col) + '1'].alignment = Alignment(horizontal='center', wrap_text=True) 
        ws1[get_column_letter(col) + '1'].value = ws1[get_column_letter(col) + '1'].value.replace('_',' ')
       

    # Assign special Yellow column headers for User Entry 
    # ws1['AB1'].fill = PatternFill("solid", start_color=HEADER_COLOR_USER_ENTRY)                
    
# Work Sheet 1 - Patient Attribution - Set Column Widths and Header Hieght
    ws1.column_dimensions['A'].width = 20
    ws1.column_dimensions['B'].width = 38
    ws1.column_dimensions['C'].width = 25
    ws1.column_dimensions['D'].width = 15
    ws1.column_dimensions['E'].width = 15
    ws1.column_dimensions['F'].width = 15
    ws1.column_dimensions['G'].width = 15
    ws1.column_dimensions['H'].width = 9
    ws1.column_dimensions['I'].width = 15
    ws1.column_dimensions['J'].width = 15      
    
# Insert rows for spreadsheet title and image
    fontStyle = Font(size = "12", bold = True)
    ws1.insert_rows(1,1)        
    for col in range(1,max_cols):
        for row in range(1,2):
            ws1[get_column_letter(col) + str(row)].alignment = Alignment(horizontal='center', vertical='top', wrap_text=True)          
            ws1[get_column_letter(col) + str(row)].font = fontStyle
            ws1[get_column_letter(col) + str(row)].fill = PatternFill("solid", fgColor=BLANK_WHITE)
    
# Insert Data Services Logo    
    img = openpyxl.drawing.image.Image(Path_to_Raw_Data + logo_file)
    img.anchor = 'F1' 
    img.width = 320
    img.height = 50
    ws1.add_image(img)    

# Set Filtering on 
    FullRange = "A2:" + get_column_letter(ws1.max_column) \
    + str(ws1.max_row)
    ws1.auto_filter.ref = FullRange    
    
# Create The Title 
    ws1.merge_cells('A1:F1')
    ws1.row_dimensions[1].height = 40 
    top_left_cell = ws1['A1']
    top_left_cell.value = "Annual Wellness Visits Provider Summary by Age"       
    
# Show last updated date after the merged cells
    today = date.today()
    top_left_cell2 = ws1['J1']
    #top_left_cell2.value = "Last Updated "
    top_left_cell2.value = today.strftime("Last Updated on %m/%d/%y")
    top_left_cell2.font = Font(size = "8", bold = False)
    
    top_left_cell.fill = PatternFill("solid", fgColor=CONTENT_TAB_COLOR)
    top_left_cell.fill = GradientFill(stop=(CONTENT_TAB_COLOR, BLANK_WHITE))
    top_left_cell.font  = Font(b=True, color="000000")
    top_left_cell.alignment = Alignment(horizontal="left", vertical="center")
    fontStyle = Font(size = "18", bold = "True")
    top_left_cell.font = fontStyle 
    
        
# Loop through the rows to set the desired number format 
    number_fmt = '_(* #,##0_);_(* (#,##0);_(* "-"_);_(@_)'    
    for col in range(5,8):
        for row in range(3,ws1.max_row + 1):
            ws1[get_column_letter(col) + str(row)].number_format = number_fmt 
            
    return work_book  

In [27]:
# organization class summary
def prettify_sheet_two(work_book):
    
    sheets = work_book.sheetnames
    ws2 = work_book[sheets[1]]
    ws2.title = "Network Summary of Annual Wellness Visits"
    ws2.sheet_properties.tabColor = CONTENT_TAB_COLOR
     
# Format the Header as bold blue centered wordwrap     
    max_cols = ws2.max_column + 1 
    for col in range(1,max_cols):
        ws2[get_column_letter(col) + '1'].font = Font(bold=True ) 
        ws2[get_column_letter(col) + '1'].fill = PatternFill("solid", start_color=HEADER_COLOR) 
        ws2[get_column_letter(col) + '1'].alignment = Alignment(horizontal='center', wrap_text=True)
        ws2[get_column_letter(col) + '1'].value = ws2[get_column_letter(col) + '1'].value.replace('_',' ')
     
    
# Work Sheet 2 - Patient Attribution - Set Column Widths and Header Hieght
    ws2.column_dimensions['A'].width = 20
    ws2.column_dimensions['B'].width = 15
    ws2.column_dimensions['C'].width = 15
    ws2.column_dimensions['D'].width = 18
    ws2.column_dimensions['E'].width = 9
    ws2.column_dimensions['F'].width = 15
    ws2.column_dimensions['G'].width = 15
    ws2.column_dimensions['H'].width = 10
    ws2.column_dimensions['I'].width = 10
    ws2.column_dimensions['J'].width = 10
       
    
# Insert rows for spreadsheet title and image
    fontStyle = Font(size = "12", bold = True)
    ws2.insert_rows(1,1)        
    for col in range(1,max_cols):
        for row in range(1,2):
            ws2[get_column_letter(col) + str(row)].alignment = Alignment(horizontal='center', vertical='top', wrap_text=True)          
            ws2[get_column_letter(col) + str(row)].font = fontStyle
            ws2[get_column_letter(col) + str(row)].fill = PatternFill("solid", fgColor=BLANK_WHITE)

# Insert Data Services Logo    
    img = openpyxl.drawing.image.Image(Path_to_Raw_Data + logo_file)
    img.anchor = 'F1' 
    img.width = 320
    img.height = 50
    ws2.add_image(img)  
    
# Set Filtering on 
    FullRange = "A2:" + get_column_letter(ws2.max_column) \
    + str(ws2.max_row)
    ws2.auto_filter.ref = FullRange    
    
# Create The Title 
    ws2.merge_cells('A1:F1')
    ws2.row_dimensions[1].height = 40 
    top_left_cell = ws2['A1']
    top_left_cell.value = "Network Summary of Annual Wellness Visits"  
    
# Show last updated date after the merged cells
    today = date.today()
    top_left_cell2 = ws2['J1']
    #top_left_cell2.value = "Last Updated "
    top_left_cell2.value = today.strftime("Last Updated on %m/%d/%y")
    top_left_cell2.font = Font(size = "8", bold = False)
    
    top_left_cell.fill = PatternFill("solid", fgColor=CONTENT_TAB_COLOR)
    top_left_cell.fill = GradientFill(stop=(CONTENT_TAB_COLOR, BLANK_WHITE))
    top_left_cell.font  = Font(b=True, color="000000")
    top_left_cell.alignment = Alignment(horizontal="left", vertical="center")
    fontStyle = Font(size = "18", bold = "True")
    top_left_cell.font = fontStyle

    # Loop through the rows to set the desired number format 
    number_fmt = '_(* #,##0_);_(* (#,##0);_(* "-"_);_(@_)'    
    for col in range(2,8):
        for row in range(3,ws2.max_row + 1):
            ws2[get_column_letter(col) + str(row)].number_format = number_fmt 
            
    return work_book  

In [28]:
# organization summary
def prettify_sheet_three(work_book):
    
    sheets = work_book.sheetnames
    ws3 = work_book[sheets[2]]
    ws3.title = "Organization (Practice) of Annual Wellness Visits"
    ws3.sheet_properties.tabColor = CONTENT_TAB_COLOR
     
# Format the Header as bold blue centered wordwrap     
    max_cols = ws3.max_column + 1 
    for col in range(1,max_cols):
        ws3[get_column_letter(col) + '1'].font = Font(bold=True ) 
        ws3[get_column_letter(col) + '1'].fill = PatternFill("solid", start_color=HEADER_COLOR) 
        ws3[get_column_letter(col) + '1'].alignment = Alignment(horizontal='center', wrap_text=True) 
        ws3[get_column_letter(col) + '1'].value = ws3[get_column_letter(col) + '1'].value.replace('_',' ')
         
    
# Work Sheet 3 - Patient Attribution - Set Column Widths and Header Hieght
    ws3.column_dimensions['A'].width = 35
    ws3.column_dimensions['B'].width = 38
    ws3.column_dimensions['C'].width = 15
    ws3.column_dimensions['D'].width = 15
    ws3.column_dimensions['E'].width = 15
    ws3.column_dimensions['F'].width = 9
    ws3.column_dimensions['G'].width = 15
    ws3.column_dimensions['H'].width = 15
 
    
# Insert rows for spreadsheet title and image
    fontStyle = Font(size = "12", bold = True)
    ws3.insert_rows(1,1)        
    for col in range(1,max_cols):
        for row in range(1,2):
            ws3[get_column_letter(col) + str(row)].alignment = Alignment(horizontal='center', vertical='top', wrap_text=True)          
            ws3[get_column_letter(col) + str(row)].font = fontStyle
            ws3[get_column_letter(col) + str(row)].fill = PatternFill("solid", fgColor=BLANK_WHITE)

# Insert Data Services Logo    
    img = openpyxl.drawing.image.Image(Path_to_Raw_Data + logo_file)
    img.anchor = 'D1' 
    img.width = 320
    img.height = 50
    ws3.add_image(img)  
    
# Set Filtering on 
    FullRange = "A2:" + get_column_letter(ws3.max_column) \
    + str(ws3.max_row)
    ws3.auto_filter.ref = FullRange  
    
# Create The Title 
    ws3.merge_cells('A1:F1')
    ws3.row_dimensions[1].height = 40 
    top_left_cell = ws3['A1']
    top_left_cell.value = "Organization (Practice) of Annual Wellness Visits"   
    
# Show last updated date after the merged cells
    today = date.today()
    top_left_cell2 = ws3['H1']
    #top_left_cell2.value = "Last Updated "
    top_left_cell2.value = today.strftime("Last Updated on %m/%d/%y")
    top_left_cell2.font = Font(size = "8", bold = False)
    
    top_left_cell.fill = PatternFill("solid", fgColor=CONTENT_TAB_COLOR)
    top_left_cell.fill = GradientFill(stop=(CONTENT_TAB_COLOR, BLANK_WHITE))
    top_left_cell.font  = Font(b=True, color="000000")
    top_left_cell.alignment = Alignment(horizontal="left", vertical="center")
    fontStyle = Font(size = "18", bold = "True")
    top_left_cell.font = fontStyle

    # Loop through the rows to set the desired number format 
    number_fmt = '_(* #,##0_);_(* (#,##0);_(* "-"_);_(@_)'    
    for col in range(3,8):
        for row in range(3,ws3.max_row + 1):
            ws3[get_column_letter(col) + str(row)].number_format = number_fmt 
            
    return work_book  

In [29]:
# provider treated lives 
def prettify_sheet_four(work_book):
    
    sheets = work_book.sheetnames
    ws4 = work_book[sheets[3]]
    ws4.title = "Provider Treated Lives of Annual Wellness Visits"
    ws4.sheet_properties.tabColor = CONTENT_TAB_COLOR
     
# Format the Header as bold blue centered wordwrap     
    max_cols = ws4.max_column + 1 
    for col in range(1,max_cols):
        ws4[get_column_letter(col) + '1'].font = Font(bold=True ) 
        ws4[get_column_letter(col) + '1'].fill = PatternFill("solid", start_color=HEADER_COLOR) 
        ws4[get_column_letter(col) + '1'].alignment = Alignment(horizontal='center', wrap_text=True)  
        ws4[get_column_letter(col) + '1'].value = ws4[get_column_letter(col) + '1'].value.replace('_',' ')
      
    
# Work Sheet 4 - Patient Attribution - Set Column Widths and Header Hieght
    ws4.column_dimensions['A'].width = 30
    ws4.column_dimensions['B'].width = 38
    ws4.column_dimensions['C'].width = 25
    ws4.column_dimensions['D'].width = 15
    ws4.column_dimensions['E'].width = 15
    ws4.column_dimensions['F'].width = 15
    ws4.column_dimensions['G'].width = 9
    ws4.column_dimensions['H'].width = 15
    ws4.column_dimensions['I'].width = 15

    
# Insert rows for spreadsheet title and image
    fontStyle = Font(size = "12", bold = True)
    ws4.insert_rows(1,1)        
    for col in range(1,max_cols):
        for row in range(1,2):
            ws4[get_column_letter(col) + str(row)].alignment = Alignment(horizontal='center', vertical='top', wrap_text=True)          
            ws4[get_column_letter(col) + str(row)].font = fontStyle
            ws4[get_column_letter(col) + str(row)].fill = PatternFill("solid", fgColor=BLANK_WHITE)

# Insert Data Services Logo    
    img = openpyxl.drawing.image.Image(Path_to_Raw_Data + logo_file)
    img.anchor = 'E1' 
    img.width = 320
    img.height = 50
    ws4.add_image(img)  

# Set Filtering on 
    FullRange = "A2:" + get_column_letter(ws4.max_column) \
    + str(ws4.max_row)
    ws4.auto_filter.ref = FullRange  
    
# Create The Title 
    ws4.merge_cells('A1:F1')
    ws4.row_dimensions[1].height = 40 
    top_left_cell = ws4['A1']
    top_left_cell.value = "Provider Treated Lives of Annual Wellness Visits"       
    
# Show last updated date after the merged cells
    today = date.today()
    top_left_cell2 = ws4['I1']
    #top_left_cell2.value = "Last Updated "
    top_left_cell2.value = today.strftime("Last Updated on %m/%d/%y")
    top_left_cell2.font = Font(size = "8", bold = False)
    
    top_left_cell.fill = PatternFill("solid", fgColor=CONTENT_TAB_COLOR)
    top_left_cell.fill = GradientFill(stop=(CONTENT_TAB_COLOR, BLANK_WHITE))
    top_left_cell.font  = Font(b=True, color="000000")
    top_left_cell.alignment = Alignment(horizontal="left", vertical="center")
    fontStyle = Font(size = "18", bold = "True")
    top_left_cell.font = fontStyle
    
    # Loop through the rows to set the desired number format 
    number_fmt = '_(* #,##0_);_(* (#,##0);_(* "-"_);_(@_)'    
    for col in range(4,9):
        for row in range(3,ws4.max_row + 1):
            ws4[get_column_letter(col) + str(row)].number_format = number_fmt 
            
    return work_book  

In [30]:
def create_sheet_feedback(work_book):
    
    # Create The Tab 
    ws_feedback = work_book.create_sheet("Feedback")
    ws_feedback.title = "Feedback or Issues" 
    ws_feedback.sheet_properties.tabColor = USER_TAB_COLOR
    
    # Blank the Canvas 
    fontStyle = Font(size = "12", bold = True)
    for col in range(1,25):
        for row in range(1,25):
            ws_feedback[get_column_letter(col) + str(row)].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)          
            ws_feedback[get_column_letter(col) + str(row)].font = fontStyle
            ws_feedback[get_column_letter(col) + str(row)].fill = PatternFill("solid", fgColor=BLANK_WHITE)

    # Create The Title 
    ws_feedback.merge_cells('A1:B1')
    ws_feedback.row_dimensions[1].height = 40
    top_left_cell = ws_feedback['A1']
    top_left_cell.value = "Feedback or Issues"

    top_left_cell.fill = PatternFill("solid", fgColor=USER_TAB_COLOR)
    top_left_cell.fill = GradientFill(stop=(USER_TAB_COLOR, BLANK_WHITE))
    top_left_cell.font  = Font(b=True, color="000000")
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    fontStyle = Font(size = "18", bold = "True")
    top_left_cell.font = fontStyle

    ws_feedback['A2'] = 'User Name'
    ws_feedback['B2'] = 'Issue or Feedback'
  
    for col in range(1,3):
        ws_feedback[get_column_letter(col) + '2'].font = Font(size = "12", bold=True ) 
        ws_feedback[get_column_letter(col) + '2'].alignment = Alignment(horizontal='center', vertical='bottom', wrap_text=True)
        ws_feedback[get_column_letter(col) + '2'].fill = PatternFill("solid", fgColor=HEADER_COLOR)
    
    ws_feedback.column_dimensions['A'].width = 30
    ws_feedback.column_dimensions['B'].width = 120 

    ws_feedback.row_dimensions[2].height = 30
 
    # Insert Data Services Logo    
    img = openpyxl.drawing.image.Image(Path_to_Raw_Data + logo_file)
    img.anchor = 'C1' 
    img.width = 320
    img.height = 50
    ws_feedback.add_image(img) 
    
    return work_book  


In [31]:
# Automate the worksheet 
def create_work_book(work_book):
    prettify_sheet_one(wb)     # Provider by Age Group  
    prettify_sheet_two(wb)     # Organization Class Summary 
    prettify_sheet_three(wb)   # Organization Summary
    prettify_sheet_four(wb)    # Provider Treated Lives    
#    create_sheet_two(wb)
#    create_sheet_three(wb)
#    create_sheet_validation(wb) 
#    create_instruction_sheet(wb)
#    create_sheet_add(wb)  
    create_sheet_feedback(wb)      
    return work_book      


In [32]:
# Pretification - Remote Process Automation - Excel Formatting - makes all the sheets formatted 
extension = 'xlsx'
os.chdir(Path_to_Excel)
files = glob.glob('*.{}'.format(extension))

for x in range(len(files)): 
    print('Processing: ' + files[x]) 
    wb = load_workbook(Path_to_Excel + files[x])
    org_name =  files[x].replace('.xlsx','')
    create_work_book(wb)
    wb.save(Path_to_Save_Excel + files[x])        # This will fail if a developer has one open 
    wb.save(Path_to_Publish_Excel + files[x])     # This will fail if a user has one open 
    print(Path_to_Save_Excel + files[x])  

Processing: Cange_Jean.xlsx
J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Annual_Wellness_Visits_ft/Cange_Jean.xlsx
Processing: Mafi_Paisley.xlsx
J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Annual_Wellness_Visits_ft/Mafi_Paisley.xlsx
Processing: Baez_Maritza.xlsx
J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Annual_Wellness_Visits_ft/Baez_Maritza.xlsx
Processing: Charles_Richard.xlsx
J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Annual_Wellness_Visits_ft/Charles_Richard.xlsx
Processing: Cleary_Kevin.xlsx
J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Annual_Wellness_Visits_ft/Cleary_Kevin.xlsx
Processing: Kuehnling_William.xlsx
J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Annual_Wellness_Visits_ft/Kuehnling_William.xlsx
Processing: Lee_Tat-Sum.xlsx
J:/OPA/GLIN Reporting Data Services Output/GPPC Scorecards/Annual_Wellness_Visits_ft/Lee_Tat-Sum.xlsx
Processing: Mitchell_Maxim.xlsx
J:/OPA/GLIN Reporting Data 

In [ ]:
# Anna STOP HERE !!!!! 

In [ ]:
# Optional - Print out the compressive List of Patient Panels 
#Org_names = patient_panel.Organization_Name.unique()
#for org in Org_names:
#        print(org)

In [ ]:
# patient_panel.describe()

In [ ]:
# test examples 
#create_organization_patient_panel("GPPC PCP Springville South Cascade")
#create_organization_patient_panel("EC PCP Erie County Medical Center FM Clinic")


In [ ]:
#Provider_panel = patient_panel[(patient_panel.Provider_Name == "ADAMS, TIMOTHY")] 

In [ ]:
#   create_work_book(wb)    

In [ ]:
# patient_panel['Gender'].value_counts()

In [ ]:
# patient_panel['Organization_Name'].value_counts()

In [ ]:
# Organization_panel.head()

In [ ]:
#wb.security.workbookPassword = 'population health'
#wb.security.lockStructure = True

In [ ]:
#spread_name = '\\Users\josep\Documents\Kaleida\'+ Organization_panel
#Organization_panel.to_excel('\\Users\josep\Documents\Kaleida\GPPC PCP Springville South Cascade.xlsx')
#"C:\Users\josep\Documents\Kaleida\Validation\AA_PCP_Alan_M_Antfleck_Physician_PLLC_Delaware_Ave.xlsx"

In [ ]:
#patient_panel.rename(columns = {'patient_age_in_years':'Patient Age'}, inplace = True)
#patient_panel.rename(columns = {'days_test':'Patient Precise Age'}, inplace = True)
#patient_panel.rename(columns = {'organization_class':'Organization_Class'}, inplace = True)
#patient_panel.rename(columns = {'organization_name':'Organization_Name'}, inplace = True)
#patient_panel.rename(columns = {'provider_name':'Provider_Name'}, inplace = True)
#patient_panel.rename(columns = {'patient_name':'Patient_Name'}, inplace = True)
#patient_panel.rename(columns = {'birth_date':'Birth_Date'}, inplace = True)
#patient_panel.rename(columns = {'patient_age_in_years':'Age'}, inplace = True)
#patient_panel.rename(columns = {'days_test':'Precise_Age'}, inplace = True)
#patient_panel.rename(columns = {'gender':'Gender'}, inplace = True)
#patient_panel.rename(columns = {'days_test':'Precise Age'}, inplace = True)


In [ ]:
# Adjust Birth Date and Calculate Age  
#now = pd.Timestamp('now')
#patient_panel['Age_in_Years'] = now - patient_panel['Birth_Date']
#patient_panel['Age_in_Years'] = patient_panel['Age_in_Years']/np.timedelta64(1,'Y')
#patient_panel['Birth_Date'] = patient_panel['Birth_Date'].dt.date
#patient_panel.round({'Age_in_Years':1})

In [ ]:
# Skip this one for AWV 
# simplify and Organize the patient Panels
#patient_panel = patient_panel[['Provider_Name','Patient_Name', 'Birth_Date', 'Age',
                               'Gender', 'Phone','Is Patient cared for by Organization (Y/N) ?', 
                               'Is Patient attributed to correct PCP (Y/N) ?',
                               'Optional Comments','Deceased', 'Deceased_Date','Flagged for Test (Y/N)',
                               'Organization_Class', 'Organization_Name',
                               'Plan_Name', 'Payer_Name', 'Age_Years','Is Patient info Correct? (Y/N)',]]

In [ ]:
# Read in the EXCEL  SQL output for Organization Provider Summary 
Organization_Provider_Summary = pd.read_excel(Path_to_Raw_Data + Raw_Data_File_Name_2)

In [ ]:
# Read in the EXCEL  SQL output for Organization Provider Summary 
Organization_Summary = pd.read_excel(Path_to_Raw_Data + Raw_Data_File_Name_3)

In [ ]:
# Optional Verify the input file structures 
#patient_panel.info()
#Organization_Provider_Summary.head()
#patient_panel.columns

In [ ]:
# Add the Provider Summary Excel Tab  
extension = 'xlsx'
os.chdir(path)
files = glob.glob('*.{}'.format(extension))
 

for x in range(len(files)): 
#    print(files[x]) 
    file_path = Path_to_Excel + files[x]
    org_name =  files[x].replace('.xlsx','')
    org_name =  org_name.replace('_',' ')    
    print (org_name)
    Provider_Summary = Organization_Provider_Summary[(Organization_Provider_Summary.Organization_Name == org_name)]  
    Provider_Summary = Provider_Summary.reset_index(drop=True)
  
    writer = pd.ExcelWriter(file_path, engine='openpyxl', mode='a')
    writer.book = load_workbook(file_path)
    Provider_Summary.to_excel(writer,sheet_name='Provider Summary', index = False)
    writer.save()
    writer.close()

In [ ]:
# The following function formats provider summary 
def create_sheet_three(work_book):
    
    sheets = work_book.sheetnames
    ws3 = work_book[sheets[2]]
    ws3.title = "Provider Summary"
    ws3.sheet_properties.tabColor = CONTENT_TAB_COLOR
     
# Format the Header as bold blue centered wordwrap     
    max_cols = ws3.max_column + 1 
    for col in range(1,max_cols):
        ws3[get_column_letter(col) + '1'].font = Font(bold=True ) 
        ws3[get_column_letter(col) + '1'].fill = PatternFill("solid", start_color=HEADER_COLOR) 
        ws3[get_column_letter(col) + '1'].alignment = Alignment(horizontal='center', wrap_text=True)    
    
# Work Sheet 1 - Patient Attribution - Set Column Widths and Header Hieght
    ws3.column_dimensions['A'].width = 30
    ws3.column_dimensions['B'].width = 30
    ws3.column_dimensions['C'].width = 30
    ws3.column_dimensions['D'].width = 30
    ws3.column_dimensions['E'].width = 30
    ws3.column_dimensions['F'].width = 30

    ws3.row_dimensions[1].height = 60    

# Replaces all underscores in header with two spaces 
    for col in range(1,6):
        ws3[get_column_letter(col) + '1'].value = ws3[get_column_letter(col) + '1'].value.replace('_','  ')
    
# Set Filtering on 
    FullRange = "A1:" + get_column_letter(ws3.max_column) \
    + str(ws3.max_row)
    ws3.auto_filter.ref = FullRange    
    
# Create The Title 
    ws_feedback.merge_cells('A1:F1')
    ws_feedback.row_dimensions[1].height = 20
    top_left_cell = ws_feedback['A1']
    top_left_cell.value = "Annual Wellness Visits Patient Detail "    


    return work_book  

In [ ]:
def create_sheet_validation(work_book):
    
    wsval = work_book.create_sheet("Validation Summary")
    wsval.title = "Validation Summary" 
    wsval.sheet_properties.tabColor = USER_TAB_COLOR
    
    wsval.merge_cells('A1:F1')
    wsval.row_dimensions[1].height = 20
    top_left_cell = wsval['A1']
    top_left_cell.value = "Validation Summary"

    top_left_cell.fill = PatternFill("solid", fgColor=USER_TAB_COLOR)
    top_left_cell.fill = fill = GradientFill(stop=(USER_TAB_COLOR, BLANK_WHITE))
    top_left_cell.font  = Font(b=True, color="000000")
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    fontStyle = Font(size = "18", bold = "True")
    top_left_cell.font = fontStyle

    wsval['A3'] = 'Number of Patients to Validate:'
    wsval['A4'] = 'Number of Patients Validate as Accurate:'
    wsval['A5'] = 'Number of Patients That are Not Validate:'
    wsval['A6'] = 'Total Validated :' 
    wsval['A7'] = 'Remaining :'
    wsval['A8'] = '% of Validated as Accurate:'

    wsval['A10'] = 'Total Distinct Providers:'
    wsval['A11'] = 'Total Patient Panel Size :'
    wsval.column_dimensions['A'].width = 50


    wsval['B3'] = '=COUNTIF(\'Patient Attribution\'!L2:L50000,"Y")'
    wsval['B4'] = '=COUNTIF(\'Patient Attribution\'!G2:G50000,"Y")'
    wsval['B5'] = '=COUNTIF(\'Patient Attribution\'!G2:G50000,"N")'
    wsval['B6'] = '=B4+B5'
    wsval['B7'] = '=B3-B6'
    wsval['B8'] = '=B4/B6'
    wsval['B8'].number_format = '0.0%'

    wsval['B10'] = '=COUNTA(\'Provider Summary\'!C2:C50000)'    
    wsval['B11'] = '=COUNTA(\'Patient Attribution\'!B2:B50000)'
    return work_book 


In [ ]:
def create_instruction_sheet(work_book):
    
    ws_instruction  = work_book.create_sheet("Instructions")
    ws_instruction.title = "Instructions" 
    ws_instruction.sheet_properties.tabColor = USER_TAB_COLOR
    
    # Blank the Canvas 
    fontStyle = Font(size = "12", bold = True)
    for col in range(1,25):
        for row in range(1,25):
            ws_instruction[get_column_letter(col) + str(row)].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)          
            ws_instruction[get_column_letter(col) + str(row)].font = fontStyle
            ws_instruction[get_column_letter(col) + str(row)].fill = PatternFill("solid", fgColor=BLANK_WHITE)
            
    for row in range(1,25):
            ws_instruction.row_dimensions[row].height = 20         
            
    ws_instruction.column_dimensions['A'].width = 3
    ws_instruction.column_dimensions['B'].width = 3
    ws_instruction.column_dimensions['C'].width = 80            
    
    
    ws_instruction.merge_cells('A1:F1')
    top_left_cell = ws_instruction['A1']
    top_left_cell.value = "Instructions for Validation"

    top_left_cell.fill = PatternFill("solid", fgColor=USER_TAB_COLOR)
    top_left_cell.fill = fill = GradientFill(stop=(USER_TAB_COLOR, BLANK_WHITE))
    top_left_cell.font  = Font(b=True, color="000000")
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    fontStyle = Font(size = "18", bold = "True")
    top_left_cell.font = fontStyle

    fontStyle = Font(size = "14", bold = "True")
    ws_instruction.merge_cells('A3:F3')
    ws_instruction['A3'].font = fontStyle
    ws_instruction['A3'] = 'Spreadsheet instructions:'

    ws_instruction['B4'] = 1
    ws_instruction['B5'] = 2 
    ws_instruction['B6'] = 3
    ws_instruction['B7'] = 4
    ws_instruction['B8'] = 5
    ws_instruction['B9'] = 6
    ws_instruction['B10'] = 7    
    ws_instruction['B11'] = 8      
    
    ws_instruction['C4'] = 'Please Filter column \'Flagged for Test\' to yes... it will randomly select 25 patients'
    ws_instruction['C5'] = 'Validation needs to be completed on the pre-selected patients and returned to the sender you received them from by February 22, 2022.'
    ws_instruction['C6'] = 'Please validate the data in the blue columns on the Patient Panel spreadsheet.'
    ws_instruction['C7'] = 'Enter a Y or N in the yellow column labeled \'Is Patient cared for by Organization ?\' '  
    ws_instruction['C8'] = 'Enter a Y or N in the yellow column labeled \'Is Patient attributed to correct PCP ?\' ' 
    ws_instruction['C9'] = 'If a N is entered, indicate why in the yellow column labeled  \'Optional Comments \' '
    ws_instruction['C10'] = 'Please review provider summary panel to ensure providers are part of organization. '     
    ws_instruction['C11'] = 'After validation is complete add or modify missing providers on the \'new provider\' tab. '   
    ws_instruction['C12'] = 'If any issues or problems occured please enter them on the \'Feedback\' tab. '       

    
    return work_book  


In [ ]:
def create_sheet_physician(work_book):
    
    wsadd = work_book.create_sheet("Adding new Physicians")
    wsadd.title = "Adding new Physicians" 
    wsadd.sheet_properties.tabColor = USER_TAB_COLOR
    
    wsadd.merge_cells('A1:G1')
    wsadd.row_dimensions[1].height = 20
    top_left_cell = wsadd['A1']
    top_left_cell.value = "Adding, Updating, or Removing Physicians"

    top_left_cell.fill = PatternFill("solid", fgColor=USER_TAB_COLOR)
    top_left_cell.fill = fill = GradientFill(stop=(USER_TAB_COLOR, BLANK_WHITE))
    top_left_cell.font  = Font(b=True, color="000000")
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    fontStyle = Font(size = "18", bold = "True")
    top_left_cell.font = fontStyle

    wsadd['A2'] = 'Provider Name'
    wsadd['B2'] = 'Provider NPI'
    wsadd['C2'] = 'Provider Credentials'
    wsadd['D2'] = 'Is Provider a Primary Care Physician PCP (Y/N) ?'
    wsadd['E2'] = 'Is Provider a Specialist (Y/N) ? '
    wsadd['F2'] = 'If Specialist enter Provider Specialty'
    wsadd['G2'] = 'Does the Provider need to be Added or Removed or Updated (A/R/U) ? '    
    wsadd['A2'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    wsadd['B2'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    wsadd['C2'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    wsadd['D2'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    wsadd['E2'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    wsadd['F2'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    wsadd['G2'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    
    for col in range(1,8):
        wsadd[get_column_letter(col) + '2'].font = Font(bold=True ) 
        wsadd[get_column_letter(col) + '2'].alignment = Alignment(horizontal='center', vertical='bottom', wrap_text=True)
        wsadd[get_column_letter(col) + '2'].fill = PatternFill("solid", fgColor=HEADER_COLOR)
    
    wsadd.column_dimensions['A'].width = 30
    wsadd.column_dimensions['B'].width = 20
    wsadd.column_dimensions['C'].width = 10
    wsadd.column_dimensions['D'].width = 20
    wsadd.column_dimensions['E'].width = 20
    wsadd.column_dimensions['F'].width = 20   
    wsadd.column_dimensions['G'].width = 20       
    
    wsadd.row_dimensions[2].height = 60
 
   
    return work_book  


In [ ]:
# Configure the code and visual settings to have consistency - Joes PC 
path = 'c:/Users/josep/Documents/Kaleida/Annual_Wellness_Visits/'
Path_to_Excel = 'c:/Users/josep/Documents/Kaleida/Annual_Wellness_Visits/'
Path_to_Save_Excel = 'c:/Users/josep/Documents/Kaleida/Annual_Wellness_Visits_ft/'
# Path_to_Share_Excel =  'https://kaleidahealth.sharefile.com/fo54e2c6-ffc7-4025-8649-443d1d382dff//'
Path_to_Raw_Data = 'c:/Users/josep/Documents/Kaleida/Annual_Wellness_Visits/'
Raw_Data_File_Name_1 = 'Annual_Wellness_Visits_Patient_Detail.xlsx'
Raw_Data_File_Name_2 = 'Organization_Provider_Summary.xlsx'
Raw_Data_File_Name_3 = 'Organization_Summary.xlsx'

print(Path_to_Raw_Data + Raw_Data_File_Name_1)
print(Path_to_Raw_Data + Raw_Data_File_Name_2)
print(Path_to_Raw_Data + Raw_Data_File_Name_3)
